In [1]:
import spark_setup
spark_setup.setup_pyspark_env()
import spark_utils

In [2]:
%%time
sc = spark_utils.get_spark_context()

Ambari - http://10.0.1.21:8080
All Applications - http://10.0.1.23:8088/cluster
CPU times: user 20 ms, sys: 8 ms, total: 28 ms
Wall time: 29.6 s


In [3]:
import pandas as pd
from pyspark.sql import SparkSession

ss = SparkSession(sc)

In [4]:
from hdfs import InsecureClient
hdfs_client = InsecureClient("http://cluster1:50070", user='hdfs')

# Load data to HDFS

https://www.kaggle.com/c/outbrain-click-prediction/data

In [5]:
import time

def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        print '%r (%r, %r) %2.2f sec' % \
              (method.__name__, args, kw, te-ts)
        return result
    return timed

In [ ]:
hdfs_client.delete("/task1", recursive=True)

False

In [ ]:
%%time
import subprocess

@timeit
def unzip_to_hdfs(fn):
    fn_out = fn.replace(".zip", "")
    print subprocess.check_output("unzip -p /data/{0} | hadoop fs -put - /task1/{1}".format(fn, fn_out), shell=True)
    
fns = [
    "clicks_test.csv.zip",
    "clicks_train.csv.zip",
    "documents_categories.csv.zip",
    "documents_entities.csv.zip",
    "documents_meta.csv.zip",
    "documents_topics.csv.zip",
    "events.csv.zip",
    "page_views.csv.zip",
    "page_views_sample.csv.zip",
    "promoted_content.csv.zip",
    "sample_submission.csv.zip"
]

for fn in fns:
    unzip_to_hdfs(fn)


'unzip_to_hdfs' (('clicks_test.csv.zip',), {}) 7.98 sec

'unzip_to_hdfs' (('clicks_train.csv.zip',), {}) 21.13 sec

'unzip_to_hdfs' (('documents_categories.csv.zip',), {}) 3.34 sec

'unzip_to_hdfs' (('documents_entities.csv.zip',), {}) 6.14 sec

'unzip_to_hdfs' (('documents_meta.csv.zip',), {}) 2.91 sec

'unzip_to_hdfs' (('documents_topics.csv.zip',), {}) 6.78 sec

'unzip_to_hdfs' (('events.csv.zip',), {}) 23.32 sec

'unzip_to_hdfs' (('page_views.csv.zip',), {}) 1358.40 sec

'unzip_to_hdfs' (('page_views_sample.csv.zip',), {}) 8.06 sec

'unzip_to_hdfs' (('promoted_content.csv.zip',), {}) 2.25 sec

'unzip_to_hdfs' (('sample_submission.csv.zip',), {}) 7.14 sec
CPU times: user 168 ms, sys: 84 ms, total: 252 ms
Wall time: 24min 7s


In [39]:
! hadoop fs -du -s -h /task1/*.csv

483.5 M  /task1/clicks_test.csv
1.4 G  /task1/clicks_train.csv
112.5 M  /task1/documents_categories.csv
309.1 M  /task1/documents_entities.csv
85.2 M  /task1/documents_meta.csv
323.7 M  /task1/documents_topics.csv
1.1 G  /task1/events.csv
88.4 G  /task1/page_views.csv
433.3 M  /task1/page_views_sample.csv
13.2 M  /task1/promoted_content.csv
260.5 M  /task1/sample_submission.csv


In [ ]:
# files are written on cluster1 node only, need to balance HDFS on cluster

In [ ]:
! hdfs dfsadmin -setBalancerBandwidth 1000000000

Balancer bandwidth is set to 1000000000


In [ ]:
%%time
! hdfs balancer -threshold 5 > balancer.log 2>&1

CPU times: user 6.48 s, sys: 2.22 s, total: 8.7 s
Wall time: 6min 21s


# Read example

In [ ]:
pvdf = ss.read.csv("/task1/page_views.csv", header=True)

In [ ]:
pvdf.dtypes

[('uuid', 'string'),
 ('document_id', 'string'),
 ('timestamp', 'string'),
 ('platform', 'string'),
 ('geo_location', 'string'),
 ('traffic_source', 'string')]

In [ ]:
pvdf.show(5)

+--------------+-----------+---------+--------+------------+--------------+
|          uuid|document_id|timestamp|platform|geo_location|traffic_source|
+--------------+-----------+---------+--------+------------+--------------+
|1fd5f051fba643|        120| 31905835|       1|          RS|             2|
|8557aa9004be3b|        120| 32053104|       1|       VN>44|             2|
|c351b277a358f0|        120| 54013023|       1|       KR>12|             1|
|8205775c5387f9|        120| 44196592|       1|       IN>16|             2|
|9cb0ccd8458371|        120| 65817371|       1|   US>CA>807|             2|
+--------------+-----------+---------+--------+------------+--------------+
only showing top 5 rows



In [ ]:
%%time
pvdf.count()

CPU times: user 76 ms, sys: 20 ms, total: 96 ms
Wall time: 9min 16s


2034275448

# Parquet is faster than CSV

http://events.linuxfoundation.org/sites/events/files/slides/ApacheCon%20BigData%20Europe%202016%20-%20Parquet%20in%20Practice%20%26%20Detail_0.pdf

In [ ]:
%%time
pvdf.write.parquet("/task1/page_views.parquet")

In [31]:
! hadoop fs -du -s -h /task1/page_views.parquet

47.3 G  /task1/page_views.parquet


In [33]:
pvdf2 = ss.read.parquet("/task1/page_views.parquet")

In [35]:
%%time
from IPython.display import display
boo = pvdf2.groupBy("geo_location").count().collect()
display(boo[:5])

[Row(geo_location=u'US>NY', count=420207),
 Row(geo_location=u'US>MS>673', count=849299),
 Row(geo_location=u'ES>07', count=139257),
 Row(geo_location=u'CO>02', count=274301),
 Row(geo_location=u'DZ', count=141209)]

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 22.2 s


In [36]:
%%time
boo = pvdf.groupBy("geo_location").count().collect()
display(boo[:5])

[Row(geo_location=u'US>MS>673', count=849299),
 Row(geo_location=u'CO>02', count=274301),
 Row(geo_location=u'DZ', count=141209),
 Row(geo_location=u'US>MT>756', count=676540),
 Row(geo_location=u'IL>01', count=21174)]

CPU times: user 96 ms, sys: 20 ms, total: 116 ms
Wall time: 10min 34s


# Convert all to Parquet

In [58]:
%%time
def convert_all_to_parquet():
    task_dir = "/task1/"
    all_files = hdfs_client.list(task_dir)
    for fn in all_files:
        if fn.endswith(".csv"):
            fn_after = fn.replace(".csv", ".parquet")
            path_before = task_dir + fn
            path_after = task_dir + fn_after
            if fn_after not in all_files:
                # generate parquet
                df = ss.read.csv(path_before, header=True)
                df.write.parquet(path_after)
            # remove csv, we have parquet now
            hdfs_client.delete(path_before)
            print fn_after, "done"

convert_all_to_parquet()

clicks_test.parquet done
clicks_train.parquet done
documents_categories.parquet done
documents_entities.parquet done
documents_meta.parquet done
documents_topics.parquet done
events.parquet done
page_views.parquet done
page_views_sample.parquet done
promoted_content.parquet done
sample_submission.parquet done
CPU times: user 96 ms, sys: 0 ns, total: 96 ms
Wall time: 4min 37s


In [60]:
! hadoop fs -du -s -h /task1/*

133.2 M  /task1/clicks_test.parquet
367.5 M  /task1/clicks_train.parquet
36.5 M  /task1/documents_categories.parquet
184.0 M  /task1/documents_entities.parquet
21.2 M  /task1/documents_meta.parquet
183.3 M  /task1/documents_topics.parquet
669.3 M  /task1/events.parquet
47.3 G  /task1/page_views.parquet
236.9 M  /task1/page_views_sample.parquet
5.0 M  /task1/promoted_content.parquet
184.2 M  /task1/sample_submission.parquet


# Preview all files

In [65]:
%%time
def preview_all_files():
    task_dir = "/task1/"
    all_files = hdfs_client.list(task_dir)
    for fn in all_files:
        df = ss.read.parquet(task_dir + fn)
        print "#" * 15 + " {0} ".format(task_dir + fn) + "#" * 15
        df.show(1)
        
preview_all_files()

############### /task1/clicks_test.parquet ###############
+----------+------+
|display_id| ad_id|
+----------+------+
|  17805143|288388|
+----------+------+
only showing top 1 row

############### /task1/clicks_train.parquet ###############
+----------+-----+-------+
|display_id|ad_id|clicked|
+----------+-----+-------+
|         1|42337|      0|
+----------+-----+-------+
only showing top 1 row

############### /task1/documents_categories.parquet ###############
+-----------+-----------+----------------+
|document_id|category_id|confidence_level|
+-----------+-----------+----------------+
|    1544588|       1513|     0.263546236|
+-----------+-----------+----------------+
only showing top 1 row

############### /task1/documents_entities.parquet ###############
+-----------+--------------------+-----------------+
|document_id|           entity_id| confidence_level|
+-----------+--------------------+-----------------+
|    1539011|e01ed0c4a3e8f8f35...|0.327269624728567|
+-----------+